# Build and Deploy a Densnet121 Model
Objective is to see whether and how to a large models in Azure

## Setup and Initialise Workspace

In [7]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.60


In [8]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: dl01
Azure region: uksouth
Subscription id: 51799227-bd67-4e34-96c2-fa93ef5da18d
Resource group: tom


## Create a Densenet121 Model

In [1]:
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.applications import imagenet_utils
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import *
import datetime

base_model = DenseNet121(include_top=False)
layer = base_model.output
layer = Dropout(0.25)(layer)
layer = GlobalAveragePooling2D()(layer)
layer = Dense(256, activation='relu', name='dense_post_pool')(layer)
layer = Dropout(0.25)(layer)
layer = Dense(2, activation='sigmoid', name='prediction')(layer)

model = Model(inputs=base_model.input,output=layer)
model.summary()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3

29089792/29084464 [==============================] - 15s 1us/step


WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

#### Convert the model to ONNX format
See [deploying-neural-network-models-to-azure-ml-service-with-keras-and-onnx](http://benalexkeen.com/deploying-neural-network-models-to-azure-ml-service-with-keras-and-onnx/)

In [14]:
import onnxmltools

onnx_model = onnxmltools.convert_keras(model) 

onnxmltools.utils.save_model(onnx_model, 'keras_densenet121.onnx')

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras2onnx/common/utils.py:42: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras2onnx/common/utils.py:42: The name tf.logging.WARN is deprecated. Please use tf.compat.v1.logging.WARN instead.

INFO - Using tensorflow=1.14.0, onnx=1.5.0, tf2onnx=1.5.2/0c735a
INFO - Using opset <onnx, 10>


### register the model in the workspace

In [17]:
from azureml.core.model import Model as AZModel

reg_model = AZModel.register(model_path = "keras_densenet121.onnx",
                       model_name = "KerasDenseNet121ONNX",
                       description = "Test Keras Model",
                       workspace = ws)

Registering model KerasDenseNet121ONNX


In [18]:
%%writefile score.py

import json
import sys

from azureml.core.model import Model
import onnxruntime
import numpy as np

def init():
    global model_path
    model_path = Model.get_model_path(model_name = 'KerasDenseNet121ONNX')

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data, dtype=np.float32)

        session = onnxruntime.InferenceSession(model_path)
        first_input_name = session.get_inputs()[0].name
        first_output_name = session.get_outputs()[0].name
        result = session.run([first_output_name], {first_input_name: data})
        # NumPy arrays are not JSON serialisable
        result = result[0].tolist()

        return {"result": result}
    except Exception as e:
        result = str(e)
        return {"error": result}

Overwriting score.py


In [19]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("onnxruntime")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [20]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "test onnx"
                                                 )

In [21]:
image = ContainerImage.create(name = "myonnxmodelimage",
                              models = [reg_model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running................................
Succeeded
Image creation operation finished for image myonnxmodelimage:1, operation "Succeeded"


In [22]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'ONNX Example')

service_name = 'onnx-example-svc'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

Running...............
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [23]:
print("Scoring API served at: {}".format(service.scoring_uri))

Scoring API served at: http://4e31d608-8107-48f3-a2a2-eea4440b51ef.uksouth.azurecontainer.io/score


### Direct register and Deploy

Create score.py

In [2]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import model_from_json

from azureml.core.model import Model

def init():
    global model
    
    model_root = Model.get_model_path('KerasDenseNet121ONNX')
    # load json and create model
    json_file = open(os.path.join(model_root, 'model.json'), 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(os.path.join(model_root, "model.h5"))   
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = np.argmax(model.predict(data), axis=1)
    return y_hat.tolist()

Writing score.py


Create myenv.xml

In [3]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('tensorflow')
cd.add_conda_package('keras')
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.60.*
- tensorflow
- keras
channels:
- conda-forge



#### Deploy to ACI
Now we can deploy. **This cell will run for about 7-8 minutes**. Behind the scene, it will do the following:
1. **Build Docker image**  
Build a Docker image using the scoring file (`score.py`), the environment file (`myenv.yml`), and the `model` object. 
2. **Register image**    
Register that image under the workspace. 
3. **Ship to ACI**    
And finally ship the image to the ACI infrastructure, start up a container in ACI using that image, and expose an HTTP endpoint to accept REST client calls.

In [11]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1, 
                                               tags={'name':'densenet121', 'framework': 'Keras'},
                                               description='densenet121-from-scratch')

In [4]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [12]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='densenet121-from-scratch',
                                       deployment_config=aciconfig,
                                       models=[reg_model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

ERROR - Models must either be of type azureml.core.Model or a str path to a file or folder.



WebserviceException: WebserviceException:
	Message: Models must either be of type azureml.core.Model or a str path to a file or folder.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Models must either be of type azureml.core.Model or a str path to a file or folder."
    }
}

In [ ]:
print(service.get_logs())

In [ ]:
print(service.scoring_uri)

### Save to file and Deploy

In [ ]:
JSON_MODEL_PATH = "./Clean121/Clean121.json"
H5_MODEL_PATH = "./Clean121/Clean121.h5"

# serialize model to JSON
model_json = model.to_json()
with open(JSON_MODEL_PATH, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(H5_MODEL_PATH)
print("Saved model to disk")


In [ ]:
DIR_MODEL_PATH = "C:/MIL_Src/data/models/caries_detector/DeepLearning/Clean121"
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = DIR_MODEL_PATH,
                       model_name = "Clean121",
                       description = "Clean Densnet121 model with extra layers",
                       workspace = ws)